In [ ]:
from requests
from dataclasses import dataclass
from dotenv import load_dotenv

load_dotenv()

from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
@dataclass
class Context:
    user_id: str

In [ ]:
@dataclass
class ResponseFormat:
    summary: str
    temperature_celcius: float
    temperature_ferenheit: float
    humidity: float

In [ ]:
@tool('get_weather', return_direct=False, description='Get the current weather for a given city')
def get_weather(city: str) -> str:
    response = requests.get('https://wttr.in/{City}?format=3'.format(City=city))
    return response.text

In [ ]:
@tool('locate_user', description='Get the location of the user based on their user ID')
def locate_user(runnable: ToolRuntime[Context]) -> str:
    match runnable.context.user_id:
        case 'user_123':
            return 'New York'
        case 'user_456':
            return 'Los Angeles'
        case 'user_789':
            return 'Los Angeles'
        case _:
            return 'Unknown'

In [ ]:
model = init_chat_model('gpt-3.5', temperature=0.6)

In [ ]:
checkpointer = InMemorySaver()

In [ ]:
agent = create_agent(
    model=model,
    tools=[get_weather, locate_user],
    system_prompt="You are a helpful weather assistant.",
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

In [ ]:
config = {'configurable': {'thread_id': 1}}

In [ ]:
response = agent.invoke({
    'messages': [
        {
            'role': 'user',
            'content': 'What is the weather like?'
        }
    ]},
    config = config,
    context = Context(user_id='user_123')
)

In [ ]:
print(response['structured_response'].summary)
print(response['structured_response'].temperature_celcius)